In [98]:
import os
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd
import numpy as np
from numpy import random
from Levenshtein import distance, hamming
import sys
import pickle
from sklearn.cross_validation import train_test_split, KFold
from nltk.util import ngrams

###Read in documents

In [117]:
#Read in Mayo documents
Mayo_list = []
for article in os.listdir('/users/markregalla/desktop/metis/Project4/MayoArticles'):
    with open("/users/markregalla/desktop/metis/Project4/MayoArticles/" 
              + article, 'r') as myfile:
        stuff = myfile.read()
    myfile.close
    if stuff != '':
        Mayo_list.append(stuff)
print len(Mayo_list)
print type(Mayo_list)

35
<type 'list'>


In [118]:
#Read in Homeopathic documents
Homeo_list = []
k = 0
for article in os.listdir('/users/markregalla/desktop/metis/Project4/HomeoArticles'):
    with open('/users/markregalla/desktop/metis/Project4/HomeoArticles/' 
              + article, 'r') as myfile:
        stuff = myfile.read()
    myfile.close
    if stuff != '':
        Homeo_list.append(str(stuff.decode("ascii", "ignore")))
    k += 1
    if k == 35:  #match the number of Mayo Clinic articles
        break
print len(Homeo_list)
print type(Homeo_list)

35
<type 'list'>


###Build corpora

In [116]:
#Build a training corpus for both sources
Mayo_train_corpus = ''
Homeo_train_corpus = ''

i = 0
while i < len(Mayo_list):
    Mayo_train_corpus = Mayo_train_corpus + Mayo_list[i]
    i += 1

i = 0
while i < len(Homeo_list):
    Homeo_train_corpus = Homeo_train_corpus + Homeo_list[i]
    i += 1

In [57]:
#Trains on first 30 documents from each source, then finds Levenshtein 
#distances of last 5 documents from each source

#Provide report on comparison of training documents
print 'Mayo test to Mayo training corpus distance:'
for i in range(30,35):
    dist = distance(Mayo_train_corpus, Mayo_list[i])
    print 'Test Distance ' + str(i) + ': ' + str(dist)

print ''

print 'Homeo test to Homeo training corpus distance:'
for i in range(30,35):
    dist = distance(Homeo_train_corpus, Homeo_list[i])
    print 'Test Distance ' + str(i) + ': ' + str(dist)

print ''

print 'Homeo test to Mayo training corpus distance:'
for i in range(30,35):
    dist = distance(Mayo_train_corpus, Homeo_list[i])
    print 'Test Distance ' + str(i) + ': ' + str(dist)

print ''

print 'Mayo test to Homeo training corpus distance:'
for i in range(30,35):
    dist = distance(Homeo_train_corpus, Mayo_list[i])
    print 'Test Distance ' + str(i) + ': ' + str(dist)


"\n#Provide report on comparison of training documents\nprint 'Mayo test to Mayo training corpus distance:'\nfor i in range(30,35):\n    dist = distance(Mayo_train_corpus, Mayo_list[i])\n    print 'Test Distance ' + str(i) + ': ' + str(dist)\n\nprint ''\n\nprint 'Homeo test to Homeo training corpus distance:'\nfor i in range(30,35):\n    dist = distance(Homeo_train_corpus, Homeo_list[i])\n    print 'Test Distance ' + str(i) + ': ' + str(dist)\n\nprint ''\n\nprint 'Homeo test to Mayo training corpus distance:'\nfor i in range(30,35):\n    dist = distance(Mayo_train_corpus, Homeo_list[i])\n    print 'Test Distance ' + str(i) + ': ' + str(dist)\n\nprint ''\n\nprint 'Mayo test to Homeo training corpus distance:'\nfor i in range(30,35):\n    dist = distance(Homeo_train_corpus, Mayo_list[i])\n    print 'Test Distance ' + str(i) + ': ' + str(dist)\n"

In [ ]:
'''
strategery: make seperate mayo and homeo training corpora
find distance of each document in that corpus to the corpus
average these distances
test document should be closer to that average distance for proper classification
'''

###Train/test splitting

In [135]:
#Test/train split on documents
Mayo_array = np.array(Mayo_list)
Mayo_labels = np.array(['Mayo' for x in range(len(Mayo_list))])
x_train_Mayo, x_test_Mayo, y_train_Mayo, y_test_Mayo = train_test_split(Mayo_array,
                                Mayo_labels, test_size=0.25, random_state=42)

Homeo_array = np.array(Homeo_list)
Homeo_labels = np.array(['Homeo' for x in range(len(Homeo_list))])
x_train_Homeo, x_test_Homeo, y_train_Homeo, y_test_Homeo = train_test_split(Homeo_array,
                                Homeo_labels, test_size=0.25, random_state=42)

print len(x_train_Homeo), len(x_test_Homeo), len(y_train_Homeo), len(y_test_Homeo)
x_test = np.concatenate((x_test_Mayo, x_test_Homeo), axis=1)
y_test = np.concatenate((y_test_Mayo, y_test_Homeo), axis=1)
y_test

26 9 26 9


array(['Mayo', 'Mayo', 'Mayo', 'Mayo', 'Mayo', 'Mayo', 'Mayo', 'Mayo',
       'Mayo', 'Homeo', 'Homeo', 'Homeo', 'Homeo', 'Homeo', 'Homeo',
       'Homeo', 'Homeo', 'Homeo'], 
      dtype='|S5')

In [86]:
#Cross validation split on documents

Mayo_array = np.array(Mayo_list)
Mayo_labels = np.array(['Mayo' for x in range(len(Mayo_list))])
x_train_Mayo = []
y_train_Mayo = []
x_test_Mayo = []
y_test_Mayo = []

kf = KFold(35, n_folds=7)
for train_index, test_index in kf:
    #x_train = Mayo_array[train_index]
    #x_test = Mayo_array[test_index]
    #y_train = Mayo_labels[train_index]
    #y_test = Mayo_labels[test_index]
    x_train_Mayo.append(Mayo_array[train_index])
    y_train_Mayo.append(Mayo_labels[train_index])
    x_test_Mayo.append(Mayo_array[test_index])
    y_test_Mayo.append(Mayo_labels[test_index])
    
Homeo_array = np.array(Homeo_list)
Homeo_labels = np.array(['Homeo' for x in range(len(Homeo_list))])
x_train_Homeo = []
y_train_Homeo = []
x_test_Homeo = []
y_test_Homeo = []

kf = KFold(35, n_folds=7)
for train_index, test_index in kf:
    #x_train = Mayo_array[train_index]
    #x_test = Mayo_array[test_index]
    #y_train = Mayo_labels[train_index]
    #y_test = Mayo_labels[test_index]
    x_train_Homeo.append(Homeo_array[train_index])
    y_train_Homeo.append(Homeo_labels[train_index])
    x_test_Homeo.append(Homeo_array[test_index])
    y_test_Homeo.append(Homeo_labels[test_index])
    
print len(x_train_Mayo), len(y_train_Mayo)
print len(x_train_Homeo), len(y_train_Homeo)
print len(x_test_Mayo), len(y_test_Mayo)
x_test = zip(y_test_Mayo, y_test_Homeo)
#print x_test_Mayo, y_test_Mayo
print x_test[0][1]
print len(x_test)
the_dist = distance(str(x_train[0][0]), str(x_test[0][1]))
print the_dist

#result_list = doc_classifier(Mayo_train_corpus, Homeo_train_corpus, x_train_Mayo,
#                             x_train_Homeo, y_train_Mayo, y_train_Homeo,
#                             x_test, y_test)

7 7
7 7
7 7
['Homeo' 'Homeo' 'Homeo' 'Homeo' 'Homeo']
7
1131


###(Make training averages manually)

In [6]:
#Generate lists of average of each document's distance to the corpus
#Takes a while, unpickle if you need the results again
Mayo_dist_list = []
for i in range(len(x_train_Mayo)):
    dist = distance(x_train_Mayo[i], Mayo_train_corpus)
    Mayo_dist_list.append(dist)
    sys.stdout.write("\r" + str(i)) 

Homeo_dist_list = []
for i in range(len(x_train_Homeo)):
    dist = distance(x_train_Homeo[i], Homeo_train_corpus)
    Homeo_dist_list.append(dist)
    sys.stdout.write("\r" + str(i))

print Mayo_dist_list
print Homeo_dist_list

27[36645, 36979, 36165, 35918, 36175, 36433, 36147, 36714, 36522, 36906, 36540, 35954, 36770, 36477, 36672, 36868, 36787, 36489, 36033, 36900, 36557, 36775, 35710, 36159, 36330, 36743, 36581, 36824]
[741312, 698054, 711261, 717227, 737927, 704019, 729752, 710556, 748280, 749779, 738161, 748691, 695484, 749659, 739214, 702475, 745263, 746409, 715791, 741390, 745688, 728466, 744161, 745175, 752936, 737117, 742878, 743057]


In [143]:
#Mayo_mean = np.mean(Mayo_dist_list, dtype=int)
#Homeo_mean = np.mean(Homeo_dist_list, dtype=int)
print Mayo_mean
print Homeo_mean

36489
65110


###Classification function

In [140]:
#Classify remaining documents to see if it is close to appropriate average
def doc_classifier(corpus1, corpus2, x_train_1, x_train_2, y_train_1, y_train_2,
                   x_test, y_test, dist1_mean=0, dist2_mean=0):
    assert type(corpus1), type(corpus2) == 'str'
    assert type(x_train_1), type(x_train_2) == 'list'
    assert type(y_train_1), type(y_train_2) == 'list'
    assert type(x_test), type(y_test) == 'list'
    assert len(x_train_1) == len(x_train_2)
    
    #Find average of distances of x_train_1 to corpus1
    if dist1_mean == 0 and dist2_mean == 0:
        for i in range(len(x_train_1)):
            dist_1 = distance(corpus1, x_train_1[i])
            dist1_mean = dist1_mean + dist_1
            sys.stdout.write("\r" + '' + str(i))
            dist_2 = distance(corpus2, x_train_2[i])
            dist2_mean = dist1_mean + dist_2
            sys.stdout.write("\r" + 'Average ' + str(i + 1) + ' of ' + str(len(x_train_1))
                             + ' completed.')
        dist1_mean = dist1_mean / len(x_train_1)
        dist2_mean = dist2_mean / len(x_train_2)
    
    result_list = [['Known', 'Predicted']]
    for i in range(len(x_test)):
        print x_test
        test_dist_1 = distance(corpus1, x_test[i])
        test_dist_2 = distance(corpus2, x_test[i])
        if abs(test_dist_1 - dist1_mean) < abs(test_dist_2 - dist2_mean):
            result_list.append([y_test[i], 'From Corpus 1'])
        else:
            result_list.append([y_test[i], 'From Corpus 2'])
        sys.stdout.write("\r" + 'Test ' + str(i + 1) + ' of ' + str(len(x_test))
                         + ' completed.')
    
    return result_list, dist1_mean, dist2_mean
    
    '''    
        if abs(Mayo_dist - Mayo_mean) < abs(Homeo_dist - Homeo_mean):
            print 'Mayo Clinic source'
        else:
            print 'Homeopathe Intl source'
        print 'Test Distance ' + str(i) + ': ' + str(Mayo_dist)

    print ''
    '''

###Results

In [141]:
%%time

#Single test/train split run
sing_result_list = []
sing_result_list, Mayo_mean, Homeo_mean = doc_classifier(Mayo_train_corpus, Homeo_train_corpus, x_train_Mayo,
                             x_train_Homeo, y_train_Mayo, y_train_Homeo,
                             x_test, y_test)
#sing_result_list.append(result_list)
sing_result_list

Average 26 of 26 completed.[ 'Vitamin C (ascorbic acid) is a water-soluble vitamin, which is needed by the body to form collagen in bones, cartilage, muscle, and blood vessels. Dietary sources of vitamin C include fruits and vegetables, particularly citrus fruits such as oranges.Severe deficiency of vitamin C causes scurvy. Although rare, scurvy results in severe symptoms and can cause death. People with scurvy are treated with vitamin C and should be under medical supervision.Many uses for vitamin C have been proposed, but evidence of benefit in scientific studies is lacking. In particular, research on asthma, cancer, and diabetes remains inconclusive, and a lack of benefit has been found for the prevention of cataracts or heart disease.The use of vitamin C in the prevention or treatment of colds remains controversial. Extensive research has been conducted. Overall, vitamin C lacked an effect on the development of colds and on cold symptoms. However, the duration of the cold shortened

In [142]:
sing_result_list

[['Known', 'Predicted'],
 ['Mayo', 'From Corpus 1'],
 ['Mayo', 'From Corpus 1'],
 ['Mayo', 'From Corpus 1'],
 ['Mayo', 'From Corpus 1'],
 ['Mayo', 'From Corpus 1'],
 ['Mayo', 'From Corpus 1'],
 ['Mayo', 'From Corpus 1'],
 ['Mayo', 'From Corpus 1'],
 ['Mayo', 'From Corpus 1'],
 ['Homeo', 'From Corpus 1'],
 ['Homeo', 'From Corpus 1'],
 ['Homeo', 'From Corpus 1'],
 ['Homeo', 'From Corpus 1'],
 ['Homeo', 'From Corpus 1'],
 ['Homeo', 'From Corpus 1'],
 ['Homeo', 'From Corpus 1'],
 ['Homeo', 'From Corpus 1'],
 ['Homeo', 'From Corpus 1']]

In [88]:
%%time

#Cross validation run
result_list = []
for i in range(len(x_train)):
    result_list.append(doc_classifier(Mayo_train_corpus, Homeo_train_corpus, x_train_Mayo[i],
                             x_train_Homeo[i], y_train_Mayo[i], y_train_Homeo[i],
                             x_test[i], y_test[i]))
print result_list

Test 2 of 4 completed.[[['Known', 'Predicted'], ['M', 'From Corpus 1'], ['a', 'From Corpus 1'], ['M', 'From Corpus 1'], ['a', 'From Corpus 1']], [['Known', 'Predicted'], ['M', 'From Corpus 1'], ['a', 'From Corpus 1'], ['M', 'From Corpus 1'], ['a', 'From Corpus 1']], [['Known', 'Predicted'], ['M', 'From Corpus 1'], ['a', 'From Corpus 1'], ['M', 'From Corpus 1'], ['a', 'From Corpus 1']], [['Known', 'Predicted'], ['M', 'From Corpus 1'], ['a', 'From Corpus 1'], ['M', 'From Corpus 1'], ['a', 'From Corpus 1']], [['Known', 'Predicted'], ['M', 'From Corpus 1'], ['a', 'From Corpus 1'], ['M', 'From Corpus 1'], ['a', 'From Corpus 1']], [['Known', 'Predicted'], ['M', 'From Corpus 1'], ['a', 'From Corpus 1'], ['M', 'From Corpus 1'], ['a', 'From Corpus 1']], [['Known', 'Predicted'], ['M', 'From Corpus 1'], ['a', 'From Corpus 1'], ['M', 'From Corpus 1'], ['a', 'From Corpus 1']]]
CPU times: user 2h 41min 53s, sys: 14.4 s, total: 2h 42min 7s
Wall time: 2h 42min 8s


In [113]:
result_list

[[['Known', 'Predicted'],
  ['M', 'From Corpus 1'],
  ['a', 'From Corpus 1'],
  ['M', 'From Corpus 1'],
  ['a', 'From Corpus 1']],
 [['Known', 'Predicted'],
  ['M', 'From Corpus 1'],
  ['a', 'From Corpus 1'],
  ['M', 'From Corpus 1'],
  ['a', 'From Corpus 1']],
 [['Known', 'Predicted'],
  ['M', 'From Corpus 1'],
  ['a', 'From Corpus 1'],
  ['M', 'From Corpus 1'],
  ['a', 'From Corpus 1']],
 [['Known', 'Predicted'],
  ['M', 'From Corpus 1'],
  ['a', 'From Corpus 1'],
  ['M', 'From Corpus 1'],
  ['a', 'From Corpus 1']],
 [['Known', 'Predicted'],
  ['M', 'From Corpus 1'],
  ['a', 'From Corpus 1'],
  ['M', 'From Corpus 1'],
  ['a', 'From Corpus 1']],
 [['Known', 'Predicted'],
  ['M', 'From Corpus 1'],
  ['a', 'From Corpus 1'],
  ['M', 'From Corpus 1'],
  ['a', 'From Corpus 1']],
 [['Known', 'Predicted'],
  ['M', 'From Corpus 1'],
  ['a', 'From Corpus 1'],
  ['M', 'From Corpus 1'],
  ['a', 'From Corpus 1']]]

In [112]:
bigrams = ngrams(y_test, 3)
#print bigrams[0]
for gram in bigrams:
    print gram
    

(array(['Mayo', 'Mayo', 'Mayo', 'Mayo', 'Mayo'], 
      dtype='|S4'), array(['Mayo', 'Mayo', 'Mayo', 'Mayo', 'Mayo'], 
      dtype='|S4'), array(['Mayo', 'Mayo', 'Mayo', 'Mayo', 'Mayo'], 
      dtype='|S4'))
(array(['Mayo', 'Mayo', 'Mayo', 'Mayo', 'Mayo'], 
      dtype='|S4'), array(['Mayo', 'Mayo', 'Mayo', 'Mayo', 'Mayo'], 
      dtype='|S4'), array(['Mayo', 'Mayo', 'Mayo', 'Mayo', 'Mayo'], 
      dtype='|S4'))
(array(['Mayo', 'Mayo', 'Mayo', 'Mayo', 'Mayo'], 
      dtype='|S4'), array(['Mayo', 'Mayo', 'Mayo', 'Mayo', 'Mayo'], 
      dtype='|S4'), array(['Mayo', 'Mayo', 'Mayo', 'Mayo', 'Mayo'], 
      dtype='|S4'))
(array(['Mayo', 'Mayo', 'Mayo', 'Mayo', 'Mayo'], 
      dtype='|S4'), array(['Mayo', 'Mayo', 'Mayo', 'Mayo', 'Mayo'], 
      dtype='|S4'), array(['Mayo', 'Mayo', 'Mayo', 'Mayo', 'Mayo'], 
      dtype='|S4'))
(array(['Mayo', 'Mayo', 'Mayo', 'Mayo', 'Mayo'], 
      dtype='|S4'), array(['Mayo', 'Mayo', 'Mayo', 'Mayo', 'Mayo'], 
      dtype='|S4'), array(['Mayo', 'Mayo', 'Mayo'

###Vectorizer

In [ ]:
vectorizer = CountVectorizer(ngram_range=(1,2))
vectorizer.fit(Mayo_list)
x = vectorizer.transform(Mayo_list)
#'humans' in vectorizer.get_feature_names()
print x

In [ ]:
x_array = x.toarray()
df = pd.DataFrame(x_array, columns=vectorizer.get_feature_names())

###Pickling

In [144]:
#with open('Mayo_dist_list.pkl', 'w') as picklefile:
#    pickle.dump(Mayo_dist_list, picklefile)
with open('corpus_averages/Mayo_mean.pkl', 'w') as picklefile:
    pickle.dump(Mayo_mean, picklefile)
with open('corpus_averages/Homeo_mean.pkl', 'w') as picklefile:
    pickle.dump(Homeo_mean, picklefile)

In [112]:
with open('Homeoo_dist_list.pkl', 'w') as picklefile:
    pickle.dump(Homeo_dist_list, picklefile)